In [6]:
alpha_name = "AlphaNetV4_LSTM"
start_date = '2019-01-01'
end_date = '2021-06-01'
change = 10

In [7]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [8]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   1%|▉                                                                                         | 61/5840 [00:00<00:12, 459.04it/s]

['AlphaNetV4_LSTM_4', 'AlphaNetV4_LSTM_3', 'AlphaNetV4_LSTM_1', 'AlphaNetV4_LSTM_7', 'AlphaNetV4_LSTM_6', 'AlphaNetV4_LSTM_2', 'AlphaNetV4_LSTM_8', 'AlphaNetV4_LSTM_9', 'AlphaNetV4_LSTM_0', 'AlphaNetV4_LSTM_5']


Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 734.41it/s]



IC table for feature "AlphaNetV4_LSTM_4" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV4_1d  |  IC_AlphaNetV4_2d  |  IC_AlphaNetV4_3d  |  IC_AlphaNetV4_4d  |  IC_AlphaNetV4_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190108-20190630 |   4.24 5.79 0.73   |   5.83 6.71 0.87   |   6.24 6.48 0.96   |   6.99 6.82 1.02   |   7.54 7.52 1.00   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   1.92 4.03 0.48   |   3.09 3.14 0.99   |   3.82 3.35 1.14   |   4.14 3.67 1.13   |   4.32 3.45 1.25   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |   3.30 5.04 0.65   |   3.82 5.63

...


AlphaNetV4_LSTM_0 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     210     |  11.53   |   6.28    |   24.97    |   3.81   |    61.54    | 3.90(20190109-20190131) |  1.21(20190131)   |   14.46    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     213     |   0.32   |   4.33    |    0.61    |   0.15   |    55.20    | 2.71(20191018-20191125) |  0.8

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 584/584 [00:00<00:00, 753.88it/s]



IC table for feature "AlphaNetV4_LSTM_5" (mean std IR):
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      period       |  IC_AlphaNetV4_1d  |  IC_AlphaNetV4_2d  |  IC_AlphaNetV4_3d  |  IC_AlphaNetV4_4d  |  IC_AlphaNetV4_5d  |
+===================+====================+====================+====================+====================+====================+
| 20190109-20190630 |   4.70 6.33 0.74   |   6.32 5.49 1.15   |   7.47 4.87 1.53   |   7.76 6.61 1.17   |   7.87 6.24 1.26   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20190701-20191231 |   3.13 2.22 1.41   |   4.67 2.58 1.81   |   6.03 2.81 2.15   |   5.70 3.49 1.63   |   6.24 3.54 1.76   |
+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 20200101-20200630 |   2.82 6.01 0.47   |   3.82 7.27

Getting BT result for date: 20210531...: 100%|██████████████████████████████████████████████████████████████████████| 584/584 [00:06<00:00, 90.07it/s]



AlphaNetV4_LSTM_5 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period       |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+==========================+===================+============+
| 20190102-20190630 |     210     |   5.71   |   5.45    |   12.02    |   2.23   |    59.83    | 3.04(20190108-20190131)  |  0.86(20190131)   |    8.53    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+--------------------------+-------------------+------------+
| 20190701-20191231 |     214     |   2.65   |   4.28    |    5.12    |   1.23   |    52.80    | 2.36(20190916-20191125)  